# First predict and select the images to be manually labeled/reviewed

In [ ]:
# ! cd /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils
# ! sh targetted_training.sh /content/drive/MyDrive/vork/ML/trees/train_data_folder/ /content/drive/MyDrive/vork/ML/trees/train_data_to_review/ 20
! cd /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils ; \
sh /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils/targetted_training.sh \
/content/drive/MyDrive/vork/ML/trees/training/plaiul/crops/ \
/content/drive/MyDrive/vork/ML/trees/training/plaiul/preds/ \
30


# Manually label the selected samples via the LblImg tool
In order to do this:
1. Link the selected images from drive, to your computer and start LblImg in that folder.
https://www.google.com/drive/download/
1. Manually label/review them via LblImg.  
This will simultaneously update the labels on drive.
Enter the
```
/content/drive/MyDrive/vork/ML/trees/train_data_to_review/
``` folder and run labelImg to open LabelImg in this folder, then start correcting the predictions.
1. When finished, go to next step: Train on new samples.

# Install dependecies... relevant in colab

In [ ]:
! pip install -r /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/requirements.txt
! pip install --upgrade opencv-python setuptools==59.5.0 albumentations==1.0.3

# Train on the output from LabelImg
! Split into Train & Valid

In [ ]:
# make sure they were maually corrected before running these cells
from pathlib import Path

preds_path = Path('/content/drive/MyDrive/vork/ML/trees/training/plaiul/preds/')

train_csv_path = preds_path.parent / 'train' / 'labels.csv'
valid_csv_path = preds_path.parent / 'valid' / 'labels.csv'

Prepare the csvs from the xmls

In [ ]:
cd /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils

In [ ]:
from glob import glob
from tqdm import tqdm
import pandas as pd
# internal import
import xml_utils

def extract_labels_as_csvs(from_folder_path, to_file):
  xmls_paths = sorted(glob(f"{str(from_folder_path)}/*.xml"))
  accumulator_bboxes_dfs = []
  for xml_path_str in tqdm(xmls_paths, desc="Converting xmls to csv for train eval"):
    xml_path = Path(xml_path_str)
    xml_as_df = xml_utils.xml_to_annotations(str(xml_path))
    accumulator_bboxes_dfs.append(xml_as_df)
  folder_bboxes_df = pd.concat(accumulator_bboxes_dfs)
  folder_bboxes_df.to_csv(to_file, index=False)

manually create divide the manually tagged images into train/valid folders, then run the following cell

In [ ]:
extract_labels_as_csvs(train_csv_path.parent, train_csv_path)
extract_labels_as_csvs(valid_csv_path.parent, valid_csv_path)

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
#load the modules
import os
import time
import numpy as np
from pathlib import Path

import torch
from deepforest import main 
# from deepforest import get_data
# from deepforest import utilities
# from deepforest import preprocess

In [ ]:
m = main.deepforest()
# alternative2
# m.use_release()
# alternative1
m.model.load_state_dict(
    torch.load(
        '/content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/models/model1/deepforest_p5863_r6672'
    )
)
m.config["train"]['epochs'] = 33
m.config["batch_size"] = 3
m.config["save-snapshot"] = False
m.config["train"]["csv_file"] = str(train_csv_path)
m.config["train"]["root_dir"] = str(train_csv_path.parent)
m.config["validation"]["csv_file"] = str(valid_csv_path)
m.config["validation"]["root_dir"] = str(valid_csv_path.parent)
m.config["gpus"] = 1

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


In [ ]:
m.create_trainer()
m.trainer.fit(m)
m.evaluate(csv_file=m.config["validation"]["csv_file"], root_dir=m.config["validation"]["root_dir"])

In [ ]:
# decomment and use when predicting on plaiul fagului
#predicted_raster = model.predict_tile(raster_path, return_plot = True, patch_size=400,patch_overlap=0.05)

In [ ]:
save_path = Path(
    '/content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/models/model1/'
)
m.trainer.save_checkpoint(save_path/"checkpoint.pl")
torch.save(m.model.state_dict(),
           save_path/'deepforest_p5933_r6815')

# Plaiul fagului

In [ ]:
# rotate tif image in mac 30°
# sips -r 30 plaiul_2.tif -o plaiul_rotated30.tif

In [ ]:
# import rasterio
# plaiul_sample = rasterio.open('/content/drive/MyDrive/vork/ML/trees/training/plaiul/plaiul_2.tif').read()
# plaiul_sample.shape

In [ ]:
# Divide a large tile into smaller arrays. Each crop will be saved to file.
# Parameters:	
# numpy_image – a numpy object to be used as a raster, usually opened from rasterio.open.read()
# path_to_raster – (str): Path to a tile that can be read by rasterio on disk
# annotations_file (str) – Path to annotations file (with column names) data in the format -> image_path, xmin, ymin, xmax, ymax, label
# base_dir (str) – Where to save the annotations and image crops relative to current working dir
# patch_size (int) – Maximum dimensions of square window
# patch_overlap (float) – Percent of overlap among windows 0->1
# allow_empty – If True, include images with no annotations to be included in the dataset
# image_name (str) – If numpy_image arg is used, what name to give the raster?
# Returns:	
# A pandas dataframe with annotations file for training.

from deepforest import preprocess
preprocess.split_raster(annotations_file = '/content/drive/MyDrive/vork/ML/trees/training/plaiul/empty_annotations.csv',
                        path_to_raster='/content/drive/MyDrive/vork/ML/trees/training/plaiul/plaiul_rotated30.tif',
                        base_dir='/content/drive/MyDrive/vork/ML/trees/training/plaiul/crops',
                        patch_size=400,
                        patch_overlap=0.05,  # buffer percentage of patch_size. (patch_overlap * patch_size) should equal the size of a tree
                        allow_empty=True,  # allow empty, since plaieul fagului is not yet annotated
                        )#image_name=None)


Predict & extract 30 most unsure samples

In [ ]:
! cd /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils ; \
sh /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils/targetted_training.sh \
/content/drive/MyDrive/vork/ML/trees/training/plaiul/crops/ \
/content/drive/MyDrive/vork/ML/trees/training/plaiul/preds/ \
30

Drive connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
